## Convert a periapsis state in the database to the parametrized variables
#### Data we want 
1. $\alpha$: SC-Earth line's angle w.r.t. Sun-B1 line
2. $\beta$: velocity ratio in <b>Earth inertial</b> frame i.e., (V_circ + DV) / V_circ
    *assumption: initial DV is parallel to V_circ when leaving LEO.

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using JSON

In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
# using R3BP
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dyanmics_parameters()

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.07480000059599223, 0.01709689063726318, 7.601281331451572)

In [9]:
# sample data
ϕ0 = 0.0
ϵr = 1.00E-06
ϵv = 1.00E-06
θsf = 3.141592654
ra_kep = 5.357114163
rp = 0.027835119
dt1 = 13.935
dt2 = 9.394115009
ra_state = [387.5004415234835, 4.0253946191092815, 6.789181649232e-23, 0.10076717389510526, 0.03283640579320107, 2.4278753237168335e-23, 1.1328266033657977]
rp_state = [388.8536029523753, -0.013186534987763569, 0.0, 5.439658762996503, 6.404830718852878, 0.0]
tof = 23.32911501


23.32911501

In [15]:
θs0 = θsf - param3b.oms * tof
θm0 = π - θs0

rE = [
    param3b.as - param3b.mu2 * cos(θm0),
    -param3b.mu2 * sin(θm0),
    0.0
]

# r_sc - r_E
vec = rp_state[1:3] - rE 
x_unit = [1.0, 0.0, 0.0]
α = acos(dot(vec, x_unit) / norm(vec))

if cross(x_unit, vec)[3] <= 0
    α = -α
end

# note that by definition, α_SB1 = α_EarthIne
α

-0.7037123162132616

In [20]:
function param_to_sb1(rp::Real, α::Real, ra::Real, θm::Real, ωm ::Real, μ2::Real, as::Real)
    # build an initial state in Earth inertial frame
    sma = (rp + ra) / 2
    v = sqrt((1-μ2) * (2/rp - 1/sma))
    
    state_i_EIne = [
        rp*cos(α), 
        rp*sin(α), 
        0,
        -v*sin(α),
        v*cos(α),
        0
    ] 
    println("state_EIne: ", state_i_EIne)
    
    # EarthIne -> Sunb1
    state_i_sb1 = SailorMoon.transform_EearthIne_to_sb1(state_i_EIne, θm, ωm, μ2, as)
    
    return state_i_sb1
end

param_to_sb1 (generic function with 1 method)

In [21]:
state = param_to_sb1(rp, α, ra_kep, θm0, param3b.oml, param3b.mu2, param3b.as)

state_EIne: [0.02122275796017492, -0.018010785499472055, 0.0, 5.437234248785317, 6.4068891630588105, 0.0]


6-element Vector{Float64}:
 388.85361920498605
  -0.013200327836297203
   0.0
   5.437234248785318
   6.4068891630588105
   0.0

In [22]:
state_Eine = SailorMoon.transform_sb1_to_EearthIne(rp_state[1:6], θm0, param3b.oml, param3b.mu2, param3b.as)

6-element Vector{Float64}:
  0.02120650534942612
 -0.01799699265093843
  0.0
  5.439658762996503
  6.404830718852878
  0.0

In [23]:
rp_state = [388.8536029523753, -0.013186534987763569, 0.0, 5.439658762996503, 6.404830718852878, 0.0]

6-element Vector{Float64}:
 388.8536029523753
  -0.013186534987763569
   0.0
   5.439658762996503
   6.404830718852878
   0.0

In [ ]:
# convert the SB1-frame state to Earth inertial 
function transform_sb1_to_EearthIne(state_earthIne::Vector, θm::Real, ωm::Real, μ2::Real, as::Real)
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    
    # step 1: SunB1 -> EM rot
    state_ = state_earthIne + [-as, 0, 0, 0, 0, 0]

    rot_mat1 = inv([
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
        ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
        0          0         0 0       0      1
    ])
    state_emrot = rot_mat1 * state_

    # step 2: EM rot to Earth Inertial
    ωm = param3b.oml

    rot_mat2 = [
        cos_θm -sin_θm 0.0 0.0 0.0 0.0
        sin_θm cos_θm 0.0 0.0 0.0 0.0
        0.0 0.0 1.0 0.0 0.0 0.0
        -ωm*sin_θm -ωm*cos_θm 0.0 cos_θm -sin_θm 0.0
        ωm*cos_θm -ωm*sin_θm 0.0 sin_θm cos_θm 0.0
        0.0 0.0 0.0 0.0 0.0 1.0
    ]
    state_ = rot_mat2 * state_emrot
    
    state_EarthIne = state_ + [μ2*cos(θm), μ2*sin(θm), 0, 0, 0, 0]

    
    return state_EarthIne
end


In [ ]:
# test function 
state_Eine = transform_sb1_to_EearthIne(rp_state[1:6], θm0, param3b.oml, param3b.mu2, param3b.as)


state_ = SailorMoon.transform_earthIne_to_EMrot(state_Eine, θm0, param3b.oml, param3b.mu2)
state_ = SailorMoon.transform_EMrot_to_SunB1(state_, π-θm0, param3b.oms, param3b.as)

state_2 = SailorMoon.transform_EearthIne_to_sb1(state_Eine, θm0, param3b.oml, param3b.mu2, param3b.as)

println("rp_state: ", rp_state[1:6])
println("state_: ", state_)
println("state_2: ", state_2)


In [ ]:
# take the velocity ratio β
r_parking = (200+6375)/param3b.lstar
v_LEO = sqrt(param3b.mu1 / r_parking)
v_SC_dep = norm(state_Eine[4:6]) 

β = v_SC_dep / v_LEO

In [ ]:
function param_to_Earth_ini(r::Real, α::Real, β::Real, θm::Real, μ::Real)
    # build an initial state in Earth inertial frame
    state_i_EIne = [
        r*cos(α), 
        r*sin(α), 
        0,
        -β*sqrt(μ / r) * sin(α),
        β*sqrt(μ / r) * cos(α),
        0
    ] 
    println("state_EIne: ", state_i_EIne)
    
    # EarthIne -> Sunb1
    state_i_sb1 = SailorMoon.transform_EearthIne_to_sb1(state_i_EIne, θm, param3b.oml, param3b.mu2, param3b.as)
    
    return state_i_sb1
end

In [ ]:
# regenerate the trajectory arc from here
# first, in reverse way we need to recreate the initial state (at SB1 frame)
println("state_Eine: ", state_Eine)

state_i = param_to_Earth_ini(r_parking, α, β, θm0, param3b.mu1)

In [ ]:
println(sqrt(14.7^2+8.27^2) ,  " ", sqrt(16.86^2))